## fit plane f(Tpeak, log tau) = power (beamwidth needed) 要改

 ### don't use this, should just use inverse function...

In [ ]:
beamwidth = 88200

import numpy, scipy, scipy.optimize
import matplotlib
from mpl_toolkits.mplot3d import  Axes3D
from matplotlib import cm # to colormap 3D surfaces from blue to red
import matplotlib.pyplot as plt
%matplotlib

# def function(data, b, c, d, e, f):
#     x = np.array(data[0])
#     y = np.array(data[1])
#     return (b*y+c)*(x)**(d*y**2+e*y+f)

# def function(data, a, b, c, d, e, f, g, h):
#     x = np.array(data[0])
#     y = np.array(data[1])
#     return (a+b*y+h*y**2)*x**(f+g*y) + (c+d*y)*x + e

# def function(data, a, b, c, d, e, f, g, h, i, j):
#     x = np.array(data[0])
#     y = np.array(data[1])
#     return (a+b*y+h*y**2)*x**(f+g*y) + (c+d*y)*x + e +i*x**2 + j*y**2

def function(data, a, b, c, d, e, f, g):
    x = np.array(data[0])
    y = np.array(data[1])
    return (a*x**b)*(c*y**d)+e*x+f*y+g

f = plt.figure(figsize=(20.0, 20.0), dpi=20)
matplotlib.pyplot.grid(True)
axes = Axes3D(f)

points = []
for i, v in enumerate(list(Si_melting_W.keys())):
    P = [int(p) for p in power_used[v]]
    tau = [np.log(beamwidth / int(v))] * len(Dict_peak_temp[v])
    T = Dict_peak_temp[v]
    point = [list(x) for x in zip(*[tau,T,P])]
    points.append(point)


x_data = []
y_data = []
z_data = []
for velo in points:
    for item in velo:
        x_data.append(item[0]) # tau/dwell time
        y_data.append(item[1]) # T
        z_data.append(item[2]) # power
        
x_data, y_data, z_data = np.array(x_data), np.array(y_data), np.array(z_data)

mask = y_data > 400
x_data = x_data[mask]
y_data = y_data[mask]
z_data = z_data[mask]

parameters_3, covariance = curve_fit(function, [x_data, y_data], z_data,  maxfev=100000)
print(parameters_3)
model_x_data = np.linspace(min(x_data), max(x_data), 30)
model_y_data = np.linspace(min(y_data), max(y_data), 30)
X, Y = np.meshgrid(model_x_data, model_y_data)
Z = function(np.array([X, Y]), *parameters_3)

z_predicted = function([x_data, y_data], *parameters_3)
differences = np.subtract(z_data, z_predicted)
squared_differences = np.square(differences)
mse = squared_differences.mean()


axes.plot_surface(X, Y, Z, cmap=cm.hot, alpha=0.2)

# for i, v in enumerate(list(Dict_kappa_Si_avg.keys())):
#     v_data = [np.log(int(v))] * len(Dict_peak_temp[v])
#     p_data = [int(p) for p in power_used[i]]
#     t_data = Dict_peak_temp[v]
#     axes.scatter(p_data, v_data, t_data, s = 250)
axes.scatter(x_data, y_data, z_data, s=500)
    

axes.set_title('Scatter Plot')
axes.set_xlabel('ln tau/dwell time', fontsize=30, labelpad=40)
axes.set_ylabel('Tpeak - Celsius',fontsize=30, labelpad=40)
axes.set_zlabel('Power - W', fontsize=30, labelpad=40)
axes.set_ylim(400, 1400)
#axes.set_zlim(400, 1400)

axes.tick_params(axis='both', which='major', labelsize=30)

plt.show()
print(mse)

In [ ]:
import numpy as np
from scipy.optimize import leastsq

parameters_3 = np.array([ 4.58541688e+00 , 8.27490988e-04, -4.54608962e+00,  1.00031164e+00,
 -1.93052376e+00 , 2.09456701e+01 , 2.79200239e+01])

def get_power_3(dwell, Tpeak):
    lntau = np.log(dwell)
    the_input = [lntau,Tpeak]
    a, b, c, d, e, f, g = parameters_3
    
    return function(the_input, a, b, c, d, e, f, g)

In [ ]:
get_power_3(88200/352, 1064)

### inversible...

In [ ]:
# from sympy import solve
# from sympy.abc import y, x, z, a, b, c, d, e, f, g, h

# a, b, c, d, e, f, g, h=  5.30354554e+02, -9.60250760e+01, -7.98869136e+02 , 9.13980490e+01, 1.35226042e+03 , 1.11308546e+00 , 2.22079583e-02 , 3.88779414e+00

# solve(function([x,y], a, b, c, d, e, f, g, h) - z, x)

# Invertible function

In [ ]:
from sympy import solve
from sympy.abc import y, x, z, a, b, c, d, e

a, b, c, d, e= 3.35953767e-01, -4.56819427e-02, -4.19905626e+06 , 4.48100862e+01, 4.19888221e+06
solve(function([x,y], a, b, c, d, e ) - z, x)

In [ ]:
# def function(data, b, c, d, e, f):
#     x = np.array(data[0])
#     y = np.array(data[1])
#     return (b*y+c)*(x)**(d*y**2+e*y+f)

def function(data, a, b, c, d, e):
    x = np.array(data[0])
    y = np.array(data[1])
    return (a+b*y)*x**2.5 + (c+d*y) + e

f = plt.figure(figsize=(20.0, 20.0), dpi=20)
matplotlib.pyplot.grid(True)
axes = Axes3D(f)

points = []
for i, v in enumerate(list(Si_melting_W.keys())):
    P = [int(p) for p in power_used[v]]
    V = [np.log(int(v))] * len(Dict_peak_temp[v])
    T = Dict_peak_temp[v]
    point = [list(x) for x in zip(*[P,V,T])]
    points.append(point)

x_data = []
y_data = []
z_data = []
for velo in points:
    for item in velo:
        x_data.append(item[0]) # power
        y_data.append(item[1]) # velocity
        z_data.append(item[2]) # temperature
        
x_data, y_data, z_data = np.array(x_data), np.array(y_data), np.array(z_data)

mask = z_data > 400
x_data = x_data[mask]
y_data = y_data[mask]
z_data = z_data[mask]

parameters, covariance = curve_fit(function, [x_data, y_data], z_data,  maxfev=1000000)
print(parameters)
model_x_data = np.linspace(min(x_data), max(x_data), 30)
model_y_data = np.linspace(min(y_data), max(y_data), 30)
X, Y = np.meshgrid(model_x_data, model_y_data)
Z = function(np.array([X, Y]), *parameters)

z_predicted = function([x_data, y_data], *parameters)
differences = np.subtract(z_data, z_predicted)
squared_differences = np.square(differences)
mse = squared_differences.mean()


axes.plot_surface(X, Y, Z, cmap=cm.hot, alpha=0.2)

axes.scatter(x_data, y_data, z_data, s=500)
    

axes.set_title('Scatter Plot')
axes.set_xlabel('power - W', fontsize=30, labelpad=40)
axes.set_ylabel('ln(velocity) - mm/s',fontsize=30, labelpad=40)
axes.set_zlabel('peak temperature - C', fontsize=30, labelpad=40)
axes.set_zlim(400, 1400)

axes.tick_params(axis='both', which='major', labelsize=30)

plt.show()
print(mse)

In [ ]:
# len(temp_line[0])

# %matplotlib
# a,b,c,d = fit(two_lorentz, temp_line[8][5])
# plt.plot(temp_line[8][5])
# # plt.plot(np.linspace(0,149,150),temp_line[0][5] )
# plt.plot(np.linspace(0,149,150), two_lorentz(np.linspace(0,149,150),a,b,c,d))
# plt.xlabel("pixel")
# plt.ylabel("Temperature (Celsius)")
# print(a,b,c,d)

# import statistics 
# print(statistics.mean(z_data_1))
# print(statistics.mean(z_data_2))
# print(statistics.stdev(z_data_1))
# print(statistics.stdev(z_data_2))
# print(max(z_data_1))
# print(min(z_data_1))
# print(max(z_data_2))
# print(min(z_data_2))
# print(len(z_data_1))
# plt.plot(z_data_2)

## curve fit用的平面們

# def function(data, b, c, d, e, f):
#     x = np.array(data[0])
#     y = np.array(data[1])
#     return (b*y+c)*(x)**(d*y**2+e*y+f)
# def function(data, a, b, c, d, e, f, g, h):
#     x = np.array(data[0])
#     y = np.array(data[1])
#     return (a+b*y+h*y**2)*x**(f+g*y) + (c+d*y)*x + e
# def function(data, a, b, c, d, e, f, g, h, i, j):
#     x = np.array(data[0])
#     y = np.array(data[1])
#     return (a+b*y+h*y**2)*x**(f+g*y) + (c+d*y)*x + e +i*x**2 + j*y**2

# def function_sigma_time(data, a, b, c, d, e, f, g):
#     x = np.array(data[0])
#     y = np.array(data[1])
#     return (a*x**b)*(c*y**d)+e*x+f*y+g


# def function(data, a, b, c, d, e, f, g, h, i, j):
#     x = np.array(data[0])
#     y = np.array(data[1])
#     return (a+b*y+h*y**2)*x**(f+g*y) + (c+d*y)*x + e +i*x**2 + j*y**2

# def function(data, b, c, d, e, f):
#     x = np.array(data[0])
#     y = np.array(data[1])
#     return (b*y+c)*(x)**(d*y**2+e*y+f)

### 用用看ML linear regression (結果超級相近，用之前的即可)

from sklearn import linear_model
import pandas as pd
import statsmodels.api as sm

data = {'power': list(x_data),
        'lnv': list(y_data),
        'sigma1': list(z_data_1),
        'sigma2': list(z_data_2)     
        }

df = pd.DataFrame(data)

x = df[['power','lnv']]
y1 = df['sigma1']
y2 = df['sigma2']

regr = linear_model.LinearRegression()
regr.fit(x, y1)
regr_2 = linear_model.LinearRegression()
regr_2.fit(x, y2)

print('Intercept 1: \n', regr.intercept_)
print('Coefficients 1: \n', regr.coef_)
print('Intercept 2: \n', regr_2.intercept_)
print('Coefficients 2: \n', regr_2.coef_)

# with statsmodels
x = sm.add_constant(x) # adding a constant
 
model = sm.OLS(y1, x).fit()
predictions = model.predict(x) 
model_2 = sm.OLS(y2, x).fit()
predictions_2 = model_2.predict(x) 
 
print_model = model.summary()
print_model_2 = model_2.summary()
#print(print_model)
print(print_model_2)
#print(model.ssr/len(x)) # mean squared error
print(model_2.ssr/len(x)) # mean squared error

In [ ]:
# #！！
# def get_background_average(fivemm_files):

#     background_files = []
#     #for velocity in fivemm_files:
#     for i,v in enumerate(list(fivemm_files.keys())):
#         filelist = [s for s in fivemm_files[v] if "_0W_" in s]
#         background_files.append(filelist)

#     intensity_list = []
#     for velocity in background_files:
#         con = []
#         for file in velocity:   
#             inten = get_blue(file)
#             con.append(inten)    
#         intensity_list.append(con)

#     back_intensity_list = []
#     for velocity in intensity_list:
#         con = np.zeros((X_DIM,Y_DIM))
#         for inten in velocity:
#             con = con + inten
#         avg = con / len(velocity)
#         back_intensity_list.append(avg)
    
#     return back_intensity_list
